# Exercício 1 - Enunciado


1. O algoritmo estendido de Euclides (EXA) aceita dois inteiros constantes  $\,a,b>0\,$  e devolve inteiros $r,s,t\,$ tais que  $\,a*s + b*t = r\,$  e  $\,r = \gcd(a,b)\,$. 
    Para além das variáveis $\,r,s,t\,$ o código requer 3 variáveis adicionais $\,r',s',t'\,$ que representam os valores de $\,r,s,t\,$ no “próximo estado”.
    ```
    INPUT  a, b
    assume  a > 0 and b > 0
    r, r', s, s', t, t' = a, b, 1, 0, 0, 1
    while r' != 0
      q = r div r'
      r, r', s, s', t, t' = r', r − q × r', s', s − q × s', r', t − q × t' 
    OUTPUT r, s, t
    ````
    1. Construa um FOTS usando BitVector de tamanho $n$ que descreva o comportamento deste programa: identifique as variáveis do modelo, o estado inicial e a relação de transição.
    2. Considere estado de erro quando $\,r=0\,$ ou alguma das variáveis atinge o “overflow”.
        Prove que o programa nunca atinge o estado de erro
    3. Prove que a relação de Bézout $\,$ $\,a*s + b*t = r\,$ é um invariante do algoritmo.

# Exercício 1 - Solução

s é o coeficiente pelo qual 'a' multiplica
t é o coeficiente pelo qual 'b' multiplica
r é o máximo divisor comum de 'a' e 'b'

a, b > 0'\n'  
a\*s + b\*t = r = mdc(a, b)  
r = mdc(a,b)

r', s', t' representam r, s, t no próximo estado

1. Fazer função declare
2. Fazer função init
3. Fazer função trans
4.  r = 0 ou s,t overflow é estado de erro. Provar que nunca atinge estado de erro
5. Provar que a*s + b*t = r é um invariante

In [ ]:
from pysmt.shortcuts import *
from pysmt.typing import INT

In [ ]:
def declare(i, bv_width):

    bv_type = BVType(bv_width)
    state = {}
    state['s'] = Symbol('s' + str(i), bv_type)
    state['t'] = Symbol('t' + str(i), bv_type)
    state['r'] = Symbol('r' + str(i), bv_type)
    state['s_prox'] = Symbol('s_prox' + str(i), bv_type)
    state['t_prox'] = Symbol('t_prox' + str(i), bv_type)
    state['r_prox'] = Symbol('r_prox' + str(i), bv_type)
    state['q'] = Symbol('q' + str(i), bv_type)
    state['pc'] = Symbol('pc' + str(i), bv_type)

    return state

In [ ]:
def init(state, a, b, bv_width):

    A = Equals(state['pc'], BV(0, bv_width))
    B = GT(Int(a), Int(0))
    C = GT(Int(b), Int(0))
    D = Equals(state['r'], BV(a, bv_width))
    E = Equals(state['s'], BV(1, bv_width))
    F = Equals(state['t'], BV(0, bv_width))
    
    return And(A, B, C, D, E, F)

In [ ]:
def trans(curr, prox, a, b, bv_width):
           
    t01 = And(
        Equals(curr['pc'], BV(0, bv_width)),
        Equals(prox['pc'], BV(1, bv_width)),
        Equals(prox['r'], BV(a, bv_width)),
        Equals(prox['r_prox'], BV(b, bv_width)),
        Equals(prox['s'], BV(1, bv_width)),
        Equals(prox['s_prox'], BV(0, bv_width)),
        Equals(prox['t'], BV(0, bv_width)),
        Equals(curr['t'], prox['t']),
        Equals(prox['t_prox'], BV(1, bv_width)),
        Equals(curr['t_prox'], prox['t_prox']),
        Equals(curr['q'], prox['q'])
    )
    
    t14 = And(
        Equals(curr['pc'], BV(1, bv_width)),
        Equals(prox['pc'], BV(4, bv_width)),
        Equals(curr['r_prox'], BV(0, bv_width)),
        Equals(curr['r_prox'], prox['r_prox']),
        Equals(curr['r'], prox['r']),
        Equals(curr['s'], prox['s']),
        Equals(curr['s_prox'], prox['s_prox']),
        Equals(curr['t'], prox['t']),
        Equals(curr['t_prox'], prox['t_prox']),
        Equals(curr['q'], prox['q'])
    )

    t12 = And(
        Equals(curr['pc'], BV(1, bv_width)),
        Equals(prox['pc'], BV(2, bv_width)),
        Not(Equals(curr['r_prox'], BV(0, bv_width))),
        Equals(curr['r_prox'], prox['r_prox']),
        Equals(curr['r'], prox['r']),
        Equals(curr['s'], prox['s']),
        Equals(curr['s_prox'], prox['s_prox']),
        Equals(curr['t'], prox['t']),
        Equals(curr['t_prox'], prox['t_prox']),
        Equals(curr['q'], prox['q'])
    )

    t23 = And(
        Equals(curr['pc'], BV(2, bv_width)),
        Equals(prox['pc'], BV(3, bv_width)),
        Equals(prox['q'], BVUDiv(curr['r'], curr['r_prox'])),
        Equals(curr['r'], prox['r']),
        Equals(curr['r_prox'], prox['r_prox']),
        Equals(curr['s'], prox['s']),
        Equals(curr['s_prox'], prox['s_prox']),
        Equals(curr['t'], prox['t']),
        Equals(curr['t_prox'], prox['t_prox'])
    )

    t31 = And(
        Equals(curr['pc'], BV(3, bv_width)),
        Equals(prox['pc'], BV(1, bv_width)),
        Equals(prox['r'], curr['r_prox']),
        Equals(prox['r_prox'], BVSub(curr['r'], BVMul(curr['q'], curr['r_prox']))),
        Equals(prox['s'], curr['s_prox']),
        Equals(prox['s_prox'], BVSub(curr['s'], BVMul(curr['q'], curr['s_prox']))),
        Equals(prox['t'], curr['t_prox']),
        Equals(prox['t_prox'], BVSub(curr['t'], BVMul(curr['q'], curr['t_prox']))),
        Equals(prox['q'], curr['q'])
    )

    t44 = And(
        Equals(curr['pc'], BV(4, bv_width)),
        Equals(prox['pc'], BV(4, bv_width)),
        Equals(curr['r'], prox['r']),
        Equals(curr['r_prox'], prox['r_prox']),
        Equals(curr['s'], prox['s']),
        Equals(curr['s_prox'], prox['s_prox']),
        Equals(curr['t'], prox['t']),
        Equals(curr['t_prox'], prox['t_prox']),
        Equals(curr['q'], prox['q'])
    )
    
    return Or(t01, t14, t12, t23, t31, t44)

In [ ]:
def gera_traco(i, a, b, K, bv_width):
    
    states = [declare(i, bv_width) for i in range(K+1)]

    with Solver() as solver:
        
        solver.add_assertion(init(states[0], a, b, bv_width))
        for i in range(K):
            solver.add_assertion(trans(states[i], states[i+1], a, b, bv_width))
        solver.add_assertion(Equals(states[-1]['pc'], BV(4, bv_width)))
        if solver.solve():
            print(f"r: {solver.get_value(states[-1]['r']).bv_signed_value()}, s: {solver.get_value(states[-1]['s']).bv_signed_value()}, t: {solver.get_value(states[-1]['t']).bv_signed_value()}")
        else:
            print("> Not feasible.")

In [ ]:
def bmc_always(declare,init,trans,inv, a, b, K, bv_width):
    
    with Solver() as solver:
        states = [declare(i, bv_width) for i in range(K+1)]
        solver.add_assertion(init(states[0], a, b, bv_width))
        solver.add_assertion(Equals(states[-1]['pc'], BV(4, bv_width)))
        
        for k in range(K):
            if k>0:
                solver.add_assertion(trans(states[k-1], states[k], a, b, bv_width))
            
            solver.push()
            solver.add_assertion(Not(inv(states[k], bv_width)))
            
            if solver.solve():
                print(f"> Invariant does not hold for {k+1} first states. Counter-example:")
                for i,state in enumerate(states[:k+1]):
                    print(f"> State {i}: pc = {solver.get_value(state['pc']).bv_signed_value()}\nq = {solver.get_value(state['q']).bv_signed_value()}\ns = {solver.get_value(state['s']).bv_signed_value()}\nt = {solver.get_value(state['t']).bv_signed_value()}\nr = {solver.get_value(state['r']).bv_signed_value()}\ns' = {solver.get_value(state['s_prox']).bv_signed_value()}\nt' = {solver.get_value(state['t_prox']).bv_signed_value()}\nr' = {solver.get_value(state['r_prox']).bv_signed_value()}")
                return
            else:
                if k==K-1:
                    print(f"> Invariant holds for the first {K} states.")
                else:
                    solver.pop()

In [ ]:
def nonzerooverflow(state, bv_width):
    zero = SBV(0, bv_width)
    max_value = BV(2**bv_width - 1, bv_width)
    return And(BVUGT(state['r'], zero), Not(BVUGT(state['s'], max_value)), Not(BVUGT(state['t'], max_value)))

In [ ]:
def bezout(state, bv_width):
    a_bv = BV(a, bv_width)
    b_bv = BV(b, bv_width)
    return Equals(BVAdd(BVMul(a_bv, state['s']), BVMul(b_bv, state['t'])), state['r'])

In [ ]:
i = int(input('Insira o número do estado inicial: '))
bv_width = int(input('Insira o número de bits par ao BitVector: '))
K = int(input('Insira o número de estados: '))

In [ ]:
a = int(input('Insira o valor de a: '))
b = int(input('Insira o valor de b: '))

In [ ]:
gera_traco(i, a, b, K, bv_width)
bmc_always(declare, init, trans, nonzerooverflow, a, b, K, bv_width)
bmc_always(declare, init, trans, bezout, a, b, K, bv_width)